# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [3]:
if IN_COLAB:
    %pip install openai
    %pip install requests
    %pip install -U instructor
    %pip install anthropic
    
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "GHPT_Instructor"

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [4]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    %git clone "https://github.com/samgregson/GHPT-experiments"
    %pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

Load the correct SSL certificates

In [5]:
if not IN_COLAB:
    from dotenv import load_dotenv
    import os
    import ssl
    load_dotenv()
    context = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))
    print(os.environ.get("REQUESTS_CA_BUNDLE"))

C:\Users\GRE111160\AppData\Roaming\ssl\ca-certificates.cer


test SSL certificate, this should return `<Response [200]>` if not try restarting the Jupyter kernel

In [6]:
import requests
requests.get("https://api.smith.langchain.com/docs", verify='C:\\Users\\GRE111160\\AppData\\Roaming\\ssl\\ca-certificates.cer')

<Response [200]>

### OpenAI

In [7]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

In [8]:
import instructor

client_instructor = instructor.patch(client, mode=instructor.Mode.JSON)

# Evals

In [9]:
inputs = [
    "Create a cylinder with a radius of 5 and a height of 10.",
    "Generate a spiral with 10 turns and a radius of 2.",
    "Create a rectangular grid of 5 by 5 points with a spacing of 10 in the x and y directions.",
    "Generate a sphere with a radius of 10 at the origin.",
    "Create a series of 10 circles with radii increasing by 1 for each subsequent circle.",
    "Generate a 3D grid of 5x5x5 spheres with a spacing of 10 in the x, y, and z directions.",
    "Create a line from the origin to the point (10,10,10).",
    "Generate a series of 10 squares with side lengths increasing by 2 for each subsequent square.",
    "Create a cone with a base radius of 5 and a height of 10.",
    "Generate a torus with a tube radius of 2 and a path radius of 5.",
    "Create a triangular grid of 5 by 5 points with a spacing of 10 in the x and y directions.",
    "Generate a series of 10 rectangles with lengths and widths increasing by 2 for each subsequent rectangle.",
    "Create a pyramid with a base side length of 5 and a height of 10.",
    "Generate a series of 10 lines with lengths increasing by 2 for each subsequent line.",
    "Create a hexagonal grid of 5 by 5 points with a spacing of 10 in the x and y directions.",
    "Generate a series of 10 hexagons with side lengths increasing by 1 for each subsequent hexagon.",
    "Create a 3D grid of 5x5x5 cubes with a spacing of 10 in the x, y, and z directions. The size of the cubes should be determined by each cube's proximity to a random point on the grid such that at distance 0 the cube has dimensions 0 and at distance 1 the cube has dimensions 1.",
    "Generate a series of 10 cylinders with radii increasing by 1 and heights increasing by 2 for each subsequent cylinder.",
    "Create a series of 10 cones with base radii increasing by 1 and heights increasing by 2 for each subsequent cone.",
    "Generate a series of 10 pyramids with base side lengths increasing by 1 and heights increasing by 2 for each subsequent pyramid.",
]

In [12]:
from pipeline.pipeline import run_pipeline

for input in inputs:
    output = run_pipeline(client=client_instructor, user_prompt=input)
    print(output.json())
    break

{"ChainOfThought":"To add two numbers, use the Number Slider component to create two editable numbers, and then use the Addition component to add them together.","Additions":[{"Name":"Number Slider","Id":1,"Value":"0..25..100"},{"Name":"Number Slider","Id":2,"Value":"-50..25..100"},{"Name":"Addition","Id":3}],"Connections":[{"From":{"Id":1,"ParameterName":"Value"},"To":{"Id":3,"ParameterName":"A"}},{"From":{"Id":2,"ParameterName":"Value"},"To":{"Id":3,"ParameterName":"B"}}],"Advice":"Make sure to set the number sliders to the correct value"}
